In [10]:
# import externals
import math
import cv2 as cv

In [11]:
class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0

    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 50:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [32]:
# create tracker object
tracker = EuclideanDistTracker()

# call the video
cap = cv.VideoCapture("Messi_FreeKick2.mp4")

# object detection for stable camera: object detector
# subtracts moving objects from stable camera
# longer the history, more precision
# higher the varthreshold, more precision
object_detector = cv.createBackgroundSubtractorMOG2(history=100, varThreshold=15)

# once we have the captured object we need to start a loop
# for a video we must extract the frames, one after another
while True:
    # for each loop, take one frame
    ret, frame = cap.read()

    h, w, _ = frame.shape
    # extract region of interest
    roi = frame[150:900, 400:1400]
    # object detection: mask to demonstrate background subtraction
    mask = object_detector.apply(roi)
    # remove greys in mask
    _, mask = cv.threshold(mask, 120, 255, cv.THRESH_BINARY)
    # hierarchy not needed, just contours extracted
    contours,_ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    # initialize array to store detections/bounds inside
    detections = []
    # loop through contours
    for c in contours:
        # calculate area and remove small elements
        area = cv.contourArea(c)
        # remove small elements
        if area > 100 and area < 35000:
            # draw contours
            # -1 implies drawing all contours, green
            # cv.drawContours(roi, [c], -1, (0, 255, 0), 2)
            # create rectangles
            x, y, w, h = cv.boundingRect(c)

            if 20 < w < 300 and 10 < h < 200:
                detections.append([x, y, w, h])

    # object tracking
    boxes_ids = tracker.update(detections)
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        if id == 88 or id == 133 or id == 327 or id == 351 or id == 389 or id == 411 or id == 420 or id == 431 or id == 441 or id == 510:
            cv.putText(roi, str(id), (x, y-20), cv.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 2)
            cv.rectangle(roi, (x,y), (x+50, y+50), (0, 255, 0), 3)

    # show region of interest, show frame, show mask
    cv.imshow("roi", roi)
    #cv.imshow("Frame", frame)
    #cv.imshow("Mask", mask)

    key = cv.waitKey(30)
    # so we can close window
    if key == 27:
        break

cap.release()
cv.destroyAllWindows

{0: (210, 435), 1: (637, 416), 2: (359, 427), 3: (539, 440)}
{0: (210, 435), 1: (637, 416), 2: (359, 427), 3: (539, 440), 4: (731, 453), 5: (718, 147)}
{0: (210, 435), 1: (637, 416), 2: (359, 427), 3: (539, 440), 4: (731, 453), 5: (746, 126)}
{0: (210, 435), 1: (637, 416), 2: (359, 427), 3: (539, 440), 4: (731, 453), 5: (746, 126), 6: (314, 12)}
{0: (210, 435), 1: (637, 416), 2: (361, 465), 3: (539, 440), 4: (731, 453), 5: (746, 126), 6: (314, 12)}
{0: (210, 426), 1: (637, 416), 2: (361, 465), 3: (539, 440), 4: (731, 453), 5: (746, 126), 6: (314, 12)}
{0: (210, 426), 1: (637, 416), 2: (356, 427), 3: (539, 440), 4: (731, 453), 5: (746, 126), 6: (314, 12)}
{0: (210, 426), 1: (637, 416), 2: (356, 427), 3: (539, 440), 4: (743, 460), 5: (746, 126), 6: (314, 12)}
{0: (210, 426), 1: (637, 416), 2: (356, 427), 3: (539, 440), 4: (743, 460), 5: (746, 126), 6: (314, 12)}
{0: (210, 426), 1: (637, 416), 2: (356, 427), 3: (539, 440), 4: (743, 460), 5: (754, 157), 6: (314, 12), 7: (888, 175)}
{0: (21

AttributeError: 'NoneType' object has no attribute 'shape'

: 